In [1]:
import os
import socket
from gp.actors.data import (
    DataActor,
    DataActorArgs,
    DBActor,
    DBActorArgs,
    KGDBArgs,
    KGName,
)
from libactor.storage._global_storage import GlobalStorage
from experiments.config import DATA_DIR, DATABASE_DIR, LIBACTOR_STORAGE_DIR
from gpp.llm.qa_llm import ExplicitV100, Schema
from gpp.actors.qa_llm_actor import QALLMActor, QALLMActorArgs
from sm.misc.funcs import get_classpath
from libactor.cache import IdentObj
from tqdm import tqdm
from gp.actors.data import KGDB
from gp.misc.evaluation.evaluator import Evaluator
from experiments.dag import *
from experiments.misc import *

In [2]:
os.environ["HF_REMOTE"] = "http://localhost:31071"
GlobalStorage.init(LIBACTOR_STORAGE_DIR)

2025-05-04 17:21:24.802 | INFO     | libactor.storage._global_storage:init:41 - GlobalStorage: /Users/rook/workspace/projects/resm-v2/data/libactor


In [3]:
data_actor = create_data_actor()

In [4]:
from gp.actors.data import GPExample
from gp.actors.el.canreg import CanRegActor, CanRegActorArgs, OracleCanReg
from gp.entity_linking.candidate_recognition import HeuristicCanReg
from kgdata.models.ontology import Ontology
from libactor.dag import DAG, Flow, Cardinality
from libactor.cache import BackendFactory, cache
from libactor.cache.identitied_object import IdentObj
from libactor.misc import identity
from sm.misc.ray_helper import get_instance

dag = create_gpp_dag(
    {
        # "data": data_actor,
        # "table": Flow("data", identity, Cardinality.ONE_TO_MANY),
        "table": [get_table, remove_unknown_columns],
        "canreg": Flow(
            "table",
            CanRegActor(
                CanRegActorArgs(
                    clspath=get_classpath(HeuristicCanReg),
                    clsargs={},
                )
            ),
        ),
        "sm": Flow(
            ["table", "canreg"],
            QALLMActor(
                QALLMActorArgs(
                    model=get_classpath(ExplicitV100),
                    model_args={"model": "allenai/OLMo-2-1124-7B-Instruct"},
                    # model_args={"model": "google/gemma-2-9b-it", "max_new_tokens": 64},
                    # model_args={"model": "meta-llama/Meta-Llama-3.1-8B-Instruct"},
                    # model_args={"model": "meta-llama/Llama-2-7b-hf"},
                    sample_size=100,
                    seed=42,
                    can_ask_for_correction=True,
                )
            ),
        ),
    },
)

In [5]:
dataset_name = "wt250"
kgdb = data_actor.get_kgdb(dataset_name)
examples = data_actor.load_dataset(dataset_name)

2025-05-04 17:21:29.670 | DEBUG    | timer:watch_and_report:74 - DataActor.load_dataset deserialize: 0.214 seconds


In [6]:
contextfn = get_gpp_context(data_actor, dataset_name)
output = dag.par_process(
    [{"table": (ex,)} for ex in examples],
    {"sm", "table"},
    [contextfn for _ in examples],
    n_jobs=1,
)

dag parallel processing: 100%|██████████| 250/250 [00:01<00:00, 136.93it/s]


Save experiment outcomes so we can analyze them in detail later.

In [7]:
ei = 4
assert [ex.id for ex in examples] == [exout["table"][0].value.id for exout in output]
display(output[ei]["table"][0].value.id)
display(output[ei]["table"][0].value.table.table.df)
output[ei]["sm"][0].value.print(env="notebook")

'wt250__1950–51_La_Liga'

,Club,City,Stadium
0,Alcoyano,Alcoy,El Collao
1,Atlético Bilbao,Bilbao,San Mamés
2,Atlético Madrid,Madrid,Metropolitano
3,Barcelona,Barcelona,Les Corts
4,Celta,Vigo,Balaídos
5,Deportivo La Coruña,A Coruña,Riazor
6,Español,Barcelona,Sarriá
7,Lérida,Lérida,Campo de los Deportes
8,Málaga,Málaga,La Rosaleda
9,Murcia,Murcia,La Condomina


HTML(value='<pre>\n00.\t<span style="background: #b7eb8f; color: black; padding: 2px; border-radius: 3px;">[0]…

In [8]:
import serde.json

from experiments.config import DATA_DIR
from experiments.misc import get_notebook_output_file

outfile = get_notebook_output_file(DATA_DIR / "experiments", 4, ".json")
serde.json.ser(
    {x["table"][0].value.id: x["sm"][0].value.to_dict() for x in output},
    outfile,
    indent=2,
)

In [9]:
evaluator = Evaluator(kgdb.ontology.value, kgdb.pydb.entity_labels.cache())

In [10]:
ctas = evaluator.avg_cta(examples, [x["sm"][0].value for x in output])
cpas = evaluator.avg_cpa(examples, [x["sm"][0].value for x in output])

2025-05-04 17:22:00.519 | INFO     | gp.misc.evaluation.evaluator:avg_cta:104 - for copying...
	cta-p	cta-r	cta-f1
48.75,48.93,48.38
2025-05-04 17:22:00.543 | INFO     | sm.evaluation.sm_metrics:precision_recall_f1:620 - Number of permutation is: 116280.0
2025-05-04 17:22:00.543 | ERROR    | sm.evaluation.sm_metrics:precision_recall_f1:626 - Permutation explosion: got 116280.0 combinations from 1 pair groups
2025-05-04 17:22:00.544 | INFO     | sm.evaluation.sm_metrics:precision_recall_f1:632 - - [(label=http://wikiba.se/ontology#Statement, X=(#nodes=4), X_prime=(#nodes=20))]
2025-05-04 17:22:00.544 | INFO     | sm.evaluation.sm_metrics:precision_recall_f1:620 - Number of permutation is: 116280.0
2025-05-04 17:22:00.544 | ERROR    | sm.evaluation.sm_metrics:precision_recall_f1:626 - Permutation explosion: got 116280.0 combinations from 1 pair groups
2025-05-04 17:22:00.545 | INFO     | sm.evaluation.sm_metrics:precision_recall_f1:632 - - [(label=http://wikiba.se/ontology#Statement, X=(

In [ ]:
cta

CTAEvalOutput(precision=0.5, recall=0.5, f1=0.5, n_corrects=1.5, n_examples=3, n_predictions=3)